In [44]:
import pandas as pd
import awswrangler as awr
import openpyxl
import os
import shutil
import unicodedata

#EXTRACT
path_to_query = r"C:\Users\raphael.almeida\Documents\SQL\Seguro de Vida Motorista\base_seguro_de_vida.sql"

with open (path_to_query,'r') as reading:
    query_readed = reading.read()

df = awr.athena.read_sql_query(query_readed,'silver')

In [75]:


row = df['e_mail'].isnull() & df['unidade'].str.contains('UNIDADE ')
df.loc[row, 'e_mail'] = 'unidade.' + df.loc[row, 'unidade'].str.replace('UNIDADE ', '').str.lower() + '@transdesk.com.br'

df.head()


,cooperativa,unidade,cliente,conjunto,coverage,beneficio,placa,motorista,e_mail
0,Stcoop,UNIDADE MARINGÁ,FELIPE ALTOE DOS SANTOS,<NA>,21309,REPARAÇÃO A TERCEIROS,QHD1I25,<NA>,unidade.maringa@transdesk.com.br
1,Stcoop,UNIDADE MARINGÁ,FELIPE ALTOE DOS SANTOS,<NA>,21308,ASSISTÊNCIA 24 HORAS,QHD1I25,<NA>,unidade.maringa@transdesk.com.br
2,Stcoop,UNIDADE MARINGÁ,FELIPE ALTOE DOS SANTOS,<NA>,21307,RASTREADOR,QHD1I25,<NA>,unidade.maringa@transdesk.com.br
3,Stcoop,UNIDADE MARINGÁ,FELIPE ALTOE DOS SANTOS,<NA>,21306,REPARAÇÃO OU REPOSIÇÃO DO VEÍCULO,QHD1I25,<NA>,unidade.maringa@transdesk.com.br
4,Stcoop,UNIDADE MARINGÁ,FELIPE ALTOE DOS SANTOS,<NA>,21305,ANUIDADE,<NA>,<NA>,unidade.maringa@transdesk.com.br


In [52]:
#removing empty spaces
df['e_mail'] = df['e_mail'].str.replace(' ','')

#removing diacritics with unicodedata

def remove_diacritics (texto):
    if isinstance(texto,str):
        return ''.join(c for c in unicodedata.normalize('NFKD',texto) if not unicodedata.combining(c))
    return texto

df['e_mail'] = df['e_mail'].apply(remove_diacritics)

df = df.reset_index(drop=True)


In [76]:
df.head()

,cooperativa,unidade,cliente,conjunto,coverage,beneficio,placa,motorista,e_mail
0,Stcoop,UNIDADE MARINGÁ,FELIPE ALTOE DOS SANTOS,<NA>,21309,REPARAÇÃO A TERCEIROS,QHD1I25,<NA>,unidade.maringa@transdesk.com.br
1,Stcoop,UNIDADE MARINGÁ,FELIPE ALTOE DOS SANTOS,<NA>,21308,ASSISTÊNCIA 24 HORAS,QHD1I25,<NA>,unidade.maringa@transdesk.com.br
2,Stcoop,UNIDADE MARINGÁ,FELIPE ALTOE DOS SANTOS,<NA>,21307,RASTREADOR,QHD1I25,<NA>,unidade.maringa@transdesk.com.br
3,Stcoop,UNIDADE MARINGÁ,FELIPE ALTOE DOS SANTOS,<NA>,21306,REPARAÇÃO OU REPOSIÇÃO DO VEÍCULO,QHD1I25,<NA>,unidade.maringa@transdesk.com.br
4,Stcoop,UNIDADE MARINGÁ,FELIPE ALTOE DOS SANTOS,<NA>,21305,ANUIDADE,<NA>,<NA>,unidade.maringa@transdesk.com.br


Conferência original

In [73]:
df_aggregated = df.groupby(['unidade','conjunto','beneficio',]).agg(
    placas = ('placa','count'),
    motoristas = ('motorista','count')
).reset_index()


df_aggregated = df_aggregated[df_aggregated['beneficio']=='SEGURO DE VIDA']
df_aggregated[df_aggregated['placas']>df_aggregated['motoristas']]



,unidade,conjunto,beneficio,placas,motoristas


Carregando para as pastas

In [ ]:
#LOAD

path_to_save = r'C:\Users\raphael.almeida\Documents\SQL\Seguro de Vida Motorista'
file_name = 'Cobertura de Seguros Motoristas.xlsx'
final_path = os.path.join(path_to_save, file_name)
os.remove(final_path)
df.to_excel(final_path, engine='openpyxl', index = False)